# 4 simulate a scenario

change ${CORE_ROOT} to your path of core.

In [ ]:
export CORE_ROOT="${HOME}/core"

change ${PJ_ROOT} to your path of uoa-poc2.

In [ ]:
export PJ_ROOT="${HOME}/uoa-poc2"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/uoa-poc2
```

## load environment variables

load from `core`

In [ ]:
source ${CORE_ROOT}/docs/environments/azure_aks/env

load from `uoa-poc2`

In [ ]:
source ${PJ_ROOT}/docs/environments/azure_aks/env

## setup alias

In [ ]:
alias now="python -c 'import datetime; print(datetime.datetime.now().strftime(\"%Y-%m-%dT%H:%M:%S.%f+09:00\"))'"
alias iso8601="python -c 'import datetime; print(datetime.datetime.now(tz=datetime.timezone.utc).isoformat(timespec=\"seconds\"))'"

## simulate a scenario

### 0. prepare

#### prepare a command to subscribe all topics

In [ ]:
echo "mosquitto_sub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /#"

#### subscribe all topics

_Outside of this notebook_
1. open a ternminal.
1. run the above command displayed `prepare a command to subscribe all topics`.

#### start uoa_poc2_bridge

_On ROS Kinect PC_
1. open three ternminal.
1. start roscore by `roscore` (terminal 1)
1. start uoa_poc2_bridge by `roslaunch uoa_poc2_bridge uoa_poc2_bridge_single.launch` (terminal 2)

#### show a ros message for sending a command to delivery_robot_01

_On ROS Kinect PC_
1. start a terminal
1. watch the ros topic by `rostopic echo /robot_bridge/turtlebot_01/cmd`

#### show a ros message for sending a command to delivery_robot_02

_On ROS Kinect PC_
1. start a terminal
1. watch the ros topic by `rostopic echo /robot_bridge/megarover_01/cmd`

#### show log messages of `robot-controller`

_Outside of this notebook_
1. open a ternminal.
1. run below command to show log messages of `robot-controller`
> `kubectl logs -f -l app=robot-controller`

#### initialize robots

In [ ]:
DELIVERY_ROBOT_IDS=$(cat << __EOD__
[
  {
    "id": "delivery_robot_01"
  },
  {
    "id": "delivery_robot_02"
  }
]
__EOD__
)

TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
LEN=$(echo ${DELIVERY_ROBOT_IDS} | jq length)
count=0
if [ ${LEN} -gt 0 ]; then
  for i in $(seq 0 $((${LEN} - 1))); do
    id=$(echo ${DELIVERY_ROBOT_IDS} | jq .[${i}].id -r)
    echo "${id} will be initialized."
    curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" https://api.${DOMAIN}/orion/v2/entities/${id}/attrs?type=${DELIVERY_ROBOT_TYPE} -X PATCH -d @- <<__EOS__
{
  "mode": {
    "value": "standby",
    "type": "string"
  },
  "remaining_waypoints_list": {
    "type": "array",
    "value": []
  },
  "navigating_waypoints": {
    "type": "object",
    "value": {}
  },
  "current_routes": {
    "type": "array",
    "value": []
  },
  "order": {
    "type": "object",
    "value": {}
  }
}
__EOS__
  let ++count
  done
fi
echo "${count} entities were initialized."

example)

```
delivery_robot_01 will be initialized.
HTTP/1.1 204 No Content
content-length: 0
fiware-correlator: 48b434b0-01f9-11ea-b2b0-3e52937c8752
date: Fri, 08 Nov 2019 07:27:56 GMT
x-envoy-upstream-service-time: 6
server: envoy

delivery_robot_02 will be initialized.
HTTP/1.1 204 No Content
content-length: 0
fiware-correlator: 48d049f2-01f9-11ea-acd2-aaa36e462053
date: Fri, 08 Nov 2019 07:27:56 GMT
x-envoy-upstream-service-time: 9
server: envoy

2 entities were initialized.
```

#### confirm entities

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" "https://api.${DOMAIN}/orion/v2/entities/?type=${DELIVERY_ROBOT_TYPE}&attrs=mode,order,remaining_waypoints_list,navigating_waypoints,current_routes" | jq .

example)

```json
[
  {
    "id": "delivery_robot_01",
    "type": "delivery_robot",
    "mode": {
      "type": "string",
      "value": "standby",
      "metadata": {}
    },
    "order": {
      "type": "object",
      "value": {},
      "metadata": {}
    },
    "remaining_waypoints_list": {
      "type": "array",
      "value": [],
      "metadata": {}
    },
    "navigating_waypoints": {
      "type": "object",
      "value": {},
      "metadata": {}
    },
    "current_routes": {
      "type": "array",
      "value": [],
      "metadata": {}
    }
  },
  {
    "id": "delivery_robot_02",
    "type": "delivery_robot",
    "mode": {
      "type": "string",
      "value": "standby",
      "metadata": {}
    },
    "order": {
      "type": "object",
      "value": {},
      "metadata": {}
    },
    "remaining_waypoints_list": {
      "type": "array",
      "value": [],
      "metadata": {}
    },
    "navigating_waypoints": {
      "type": "object",
      "value": {},
      "metadata": {}
    },
    "current_routes": {
      "type": "array",
      "value": [],
      "metadata": {}
    }
  }
]
```

#### move delivery_robot_01 to staret position

In [ ]:
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /${DELIVERY_ROBOT_TYPE}/${DELIVERY_ROBOT_01}/cmd -m "{\"send_cmd\":{\"time\":\"$(now)\",\"cmd\":\"navi\",\"waypoints\":[{\"point\":{\"x\":-0.4,\"y\":1.5,\"z\":0},\"angle\":{\"roll\":0,\"pitch\":0,\"yaw\":-1.57}}]}}"

#### confirm mqtt message

example)

```
Client mosq-TX4bePNymjLv3bvrtN received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmd', ... (159 bytes))
{"send_cmd":{"time":"2019-11-11T12:13:41.824398+09:00","cmd":"navi","waypoints":[{"point":{"x":-0.4,"y":1.5,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-1.57}}]}}
```

#### confirm ros message for delivery_robot_01 ( `/robot_bridge/turtlebot_01/cmd` )

example)

```yaml
id: "delivery_robot_01"
type: "delivery_robot"
time: "2019-11-11T12:13:41.824398+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: -0.4
      y: 1.5
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -1.57
---
```

#### move delivery_robot_02 to staret position

In [ ]:
mosquitto_pub -h mqtt.${DOMAIN} -p 8883 --cafile ${CORE_ROOT}/secrets/DST_Root_CA_X3.pem -d -u iotagent -P ${MQTT__iotagent} -t /${DELIVERY_ROBOT_TYPE}/${DELIVERY_ROBOT_02}/cmd -m "{\"send_cmd\":{\"time\":\"$(now)\",\"cmd\":\"navi\",\"waypoints\":[{\"point\":{\"x\":-1.5,\"y\":1.5,\"z\":0},\"angle\":{\"roll\":0,\"pitch\":0,\"yaw\":-1.57}}]}}"

#### confirm mqtt message

example)

```
Client mosq-TX4bePNymjLv3bvrtN received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/cmd', ... (159 bytes))
{"send_cmd":{"time":"2019-11-11T12:16:52.974547+09:00","cmd":"navi","waypoints":[{"point":{"x":-1.5,"y":1.5,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-1.57}}]}}
```

#### confirm ros message for delivery_robot_02 ( `/robot_bridge/megarover_01/cmd` )

example)

```yaml
id: "delivery_robot_02"
type: "delivery_robot"
time: "2019-11-11T12:16:52.974547+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: -1.5
      y: 1.5
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -1.57
---
```

### 1. ship an item to a destination

#### post a shipment data

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens | map(select(.allowed_paths[] | contains ("^/controller/.*$"))) | .[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Content-type: application/json"  https://api.${DOMAIN}/controller/api/v1/shipments/ -X POST -d @-<<__EOS__
{
  "destination": {
    "name": "会議室1中"
  },
  "updated": [
    {
      "prev_quantity": 5,
      "new_quantity": 4,
      "reservation": 1,
      "title": "美味しい天然水（2L）",
      "place": "事務室"
    }
  ]
}
__EOS__

example)

```
HTTP/1.1 201 Created
server: envoy
date: Fri, 08 Nov 2019 05:38:52 GMT
content-type: application/json
content-length: 140
access-control-allow-origin: *
x-envoy-upstream-service-time: 524

{"delivery_robot":{"id":"delivery_robot_02"},"order":{"destination":"place_Jdash","source":"place_V","via":["place_L"]},"result":"success"}
```

#### confirm the MQTT message

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/cmd', ... (154 bytes))
{"send_cmd":{"time":"2019-11-08T14:38:52.897+09:00","cmd":"navi","waypoints":[{"point":{"x":0,"y":-1.5,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-0.46}}]}}
```

#### confirm ros message for delivery_robot_02 ( `/robot_bridge/megarover_01/cmd` )

example)
```yaml
id: "delivery_robot_02"
type: "delivery_robot"
time: "2019-11-08T14:38:52.897+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 0.0
      y: -1.5
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -0.46
---
```

#### confirm delivery robot entity (delivery_robot_02)

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${DELIVERY_ROBOT_02}/ | jq .

confirm below:

* `send_cmd_status` is `PENDING`.
* `current_routes`, `navigating_waypoints` and `remaining_waypoints_list` is filled.
* `navigating_waypoints.waypoints` is the same of `waypoints` of MQTT message and ros message.

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* all token is not locked. (`is_locked` is `false`)
* all `lock_owner_id` are empty.
* all `waitings` are empty list.

### 2. notify that the command is received

#### send a ros message

_On ROS Kinect PC_
1. start a terminal
1. execute below command.
```
rostopic pub -1 /robot_bridge/megarover_01/cmdexe uoa_poc2_msgs/r_result "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-08T14:00:00.000+09:00'
received_time: '2019-11-08T14:38:52.897+09:00'
received_cmd: 'navi'
received_waypoints:
  -
    point:
      x: 0.0
      y: -1.5
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -0.46
result: 'ack'
errors: ['']
"
```

#### confirm the MQTT message

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/cmdexe', ... (282 bytes))
{"send_cmd": {"received_time": "2019-11-08T14:38:52.897+09:00", "errors": [""], "received_cmd": "navi", "received_waypoints": [{"angle": {"yaw": -0.46, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.5, "x": 0.0, "z": 0.0}}], "result": "ack", "time": "2019-11-08T14:00:00.000+09:00"}}
```

#### confirm delivery robot entity (delivery_robot_02)

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${DELIVERY_ROBOT_02}/ | jq .

confirm below:
* `send_cmd_status` turns to `OK` and `send_cmd_info` has the result message.

### 3. emulate the move of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-08T14:00:00.000+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 0.0, y: -0.2, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.2}
destination:
  point: {x: 0.0, y: -1.5, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -0.46}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (530 bytes))
{"errors": [], "pose": {"angle": {"yaw": -0.2, "roll": 0.0, "pitch": 0.0}, "point": {"y": -0.2, "x": 0.0, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -0.46, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.5, "x": 0.0, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-08T15:02:14.774031+09:00"}
```

#### confirm that a MQTT message to `/robot_ui/robotui_02/cmd` is sent automatically

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_02/cmd', ... (98 bytes))
{"send_state":{"time":"2019-11-08T15:02:14.862+09:00","state":"moving","destination":"事務室"}}
```

#### confirm log messages

example)
```
update robot state, robot_id=delivery_robot_02, current_mode=standby, next_mode=navi
publish new state to robot ui(robotui_02), current_state=standby, next_state=moving, destination=事務室
```

#### send a `navi` ros message again

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-08T14:00:00.000+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 0.0, y: -0.2, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.2}
destination:
  point: {x: 0.0, y: -1.5, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -0.46}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

confirm below:

* a MQTT message to `/delivery_robot/delivery_robot_02/attrs` is sent.
* no MQTT message to `/robot_ui/robotui_02/cmd` is sent.
* no log message is shown.

### 4. emulate the stop of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-08T14:01:00.000+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 0.0, y: -1.49, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.45}
destination:
  point: {x: 0.0, y: -1.5, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -0.46}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (535 bytes))
{"errors": [], "pose": {"angle": {"yaw": -0.45, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.49, "x": 0.0, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -0.46, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.5, "x": 0.0, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-08T15:59:44.221888+09:00"}
```

#### confirm that a MQTT message to `/delivery_robot/delivery_robot_02/cmd` is sent automatically

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/cmd', ... (204 bytes))
{"send_cmd":{"time":"2019-11-08T15:59:44.476+09:00","cmd":"navi","waypoints":[{"point":{"x":0.5,"y":-2,"z":0},"angle":null},{"point":{"x":3.71,"y":-1.99,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-1.57}}]}}
```

#### confirm that a ros message for delivery_robot_02 ( `/robot_bridge/megarover_01/cmd` ) is sent automatically

example)

```yaml
id: "delivery_robot_02"
type: "delivery_robot"
time: "2019-11-08T15:59:44.476+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 0.5
      y: -2.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 3.71
      y: -1.99
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -1.57
---
```

#### confirm log messages

example)
```
update robot state, robot_id=delivery_robot_02, current_mode=navi, next_mode=standby
lock token (token_a) by delivery_robot_02
move robot(delivery_robot_02) to "place_L" (waypoints=[{'point': {'x': 0.5, 'y': -2, 'z': 0}, 'angle': None}, {'point': {'x': 3.71, 'y': -1.99, 'z': 0}, 'angle': {'roll': 0, 'pitch': 0, 'yaw': -1.57}}]
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is locked by `delivery_robot_02`, but no robot is waiting for this token yet.
* `token_b` and `token_c` is not locked.

#### send a `standby` ros message again

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-08T14:01:00.000+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 0.0, y: -1.49, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.45}
destination:
  point: {x: 0.0, y: -1.5, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -0.46}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

confirm below:

* a MQTT message to `/delivery_robot/delivery_robot_02/attrs` is sent.
* no MQTT message to `/delivery_robot/delivery_robot_02/cmd` is sent.
* no log message is shown.

### 5. emulate the move of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-08T14:02:00.000+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 0.0, y: -1.1, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.4}
destination:
  point: {x: 0.0, y: -2.0, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (493 bytes))
{"errors": [], "pose": {"angle": {"yaw": -0.4, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.1, "x": 0.0, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -2.0, "x": 0.0, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-08T16:12:47.472778+09:00"}
```

#### confirm log messages

example)
```
update robot state, robot_id=delivery_robot_02, current_mode=standby, next_mode=navi
```

### 6. ship items sotred different warehouses each other to another destination

#### post a shipment data

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens | map(select(.allowed_paths[] | contains ("^/controller/.*$"))) | .[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Content-type: application/json"  https://api.${DOMAIN}/controller/api/v1/shipments/ -X POST -d @-<<__EOS__
{
  "destination": {
    "name": "会議室2中"
  },
  "updated": [
    {
      "prev_quantity": 4,
      "new_quantity": 3,
      "reservation": 1,
      "title": "美味しい天然水（2L）",
      "place": "事務室"
    },
    {
      "prev_quantity": 10,
      "new_quantity": 8,
      "reservation": 2,
      "title": "地域名産炭酸水（500ml）",
      "place": "個室"
    }
  ]
}
__EOS__

example)

```
HTTP/1.1 201 Created
server: envoy
date: Fri, 08 Nov 2019 07:29:03 GMT
content-type: application/json
content-length: 150
access-control-allow-origin: *
x-envoy-upstream-service-time: 371

{"delivery_robot":{"id":"delivery_robot_01"},"order":{"destination":"place_Idash","source":"place_Q","via":["place_G","place_L"]},"result":"success"}
```

#### confirm the MQTT message

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmd', ... (154 bytes))
{"send_cmd":{"time":"2019-11-08T16:29:03.796+09:00","cmd":"navi","waypoints":[{"point":{"x":0,"y":-1.5,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-0.46}}]}}
```

#### confirm ros message for delivery_robot_02 ( `/robot_bridge/turtlebot_01/cmd` )

example)

```yaml
id: "delivery_robot_01"
type: "delivery_robot"
time: "2019-11-08T16:29:03.796+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 0.0
      y: -1.5
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -0.46
---
```

#### confirm delivery robot entity (delivery_robot_01)

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${DELIVERY_ROBOT_01}/ | jq .

confirm below:

* `send_cmd_status` is `PENDING`.
* `current_routes`, `navigating_waypoints` and `remaining_waypoints_list` is filled.
* `navigating_waypoints.waypoints` is the same of `waypoints` of MQTT message and ros message.

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is still locked by `delivery_robot_02`, but no robot is waiting for this token yet.
* `token_b` and `token_c` is not locked.

### 7. notify that the command is received

#### send a ros message

_On ROS Kinect PC_
1. start a terminal
1. execute below command.
```
rostopic pub -1 /robot_bridge/turtlebot_01/cmdexe uoa_poc2_msgs/r_result "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-08T14:03:00.000+09:00'
received_time: '2019-11-08T16:29:03.796+09:00'
received_cmd: 'navi'
received_waypoints:
  -
    point:
      x: 0.0
      y: -1.5
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -0.46
result: 'ack'
errors: ['']
"
```

#### confirm the MQTT message

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmdexe', ... (282 bytes))
{"send_cmd": {"received_time": "2019-11-08T16:29:03.796+09:00", "errors": [""], "received_cmd": "navi", "received_waypoints": [{"angle": {"yaw": -0.46, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.5, "x": 0.0, "z": 0.0}}], "result": "ack", "time": "2019-11-08T14:03:00.000+09:00"}}
```

#### confirm delivery robot entity (delivery_robot_01)

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/${DELIVERY_ROBOT_01}/ | jq .

confirm below:
* `send_cmd_status` turns to `OK` and `send_cmd_info` has the result message.

### 8. emulate the move of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-08T14:03:00.000+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 0.0, y: -0.2, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.2}
destination:
  point: {x: 0.0, y: -1.5, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -0.46}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (530 bytes))
{"errors": [], "pose": {"angle": {"yaw": -0.2, "roll": 0.0, "pitch": 0.0}, "point": {"y": -0.2, "x": 0.0, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -0.46, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.5, "x": 0.0, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-08T16:44:10.609862+09:00"}
```

#### confirm that a MQTT message to `/robot_ui/robotui_01/cmd` is sent automatically

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_01/cmd', ... (98 bytes))
{"send_state":{"time":"2019-11-08T16:44:10.716+09:00","state":"moving","destination":"事務室"}}
```

#### confirm log messages

example)
```
update robot state, robot_id=delivery_robot_01, current_mode=standby, next_mode=navi
publish new state to robot ui(robotui_01), current_state=standby, next_state=moving, destination=事務室
```

#### send a `navi` ros message again

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-08T14:03:00.000+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 0.0, y: -0.2, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.2}
destination:
  point: {x: 0.0, y: -1.5, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -0.46}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

confirm below:

* a MQTT message to `/delivery_robot/delivery_robot_01/attrs` is sent.
* no MQTT message to `/robot_ui/robotui_01/cmd` is sent.
* no log message is shown.

### 9. emulate the stop of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-08T14:03:00.000+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 0.0, y: -1.51, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.46}
destination:
  point: {x: 0.0, y: -1.5, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -0.46}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (535 bytes))
{"errors": [], "pose": {"angle": {"yaw": -0.46, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.51, "x": 0.0, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -0.46, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.5, "x": 0.0, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-08T16:46:38.044650+09:00"}
```

#### confirm that no MQTT message to `/delivery_robot/delivery_robot_01/cmd` and no ros message for delivery_robot_01 ( `/robot_bridge/turtlebot_01/cmd` ) is sent automatically

`token_a` has been locked by `delivery_robot_02`, so `delivery_robot_01` can not move into restlicted area

#### confirm log messages

example)
```
update robot state, robot_id=delivery_robot_01, current_mode=navi, next_mode=standby
wait token (token_a) by delivery_robot_01
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is still locked by `delivery_robot_02`, and `delivery_robot_01` is now waiting for this token.
* `token_b` and `token_c` is not locked.

#### send a `standby` ros message again

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-08T14:03:00.000+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 0.0, y: -1.51, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.46}
destination:
  point: {x: 0.0, y: -1.5, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -0.46}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

confirm below:

* a MQTT message to `/delivery_robot/delivery_robot_01/attrs` is sent.
* no MQTT message to `/delivery_robot/delivery_robot_01/cmd` is sent.
* no log message is shown.

### 10. emulate the move of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-08T14:04:00.000+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 0.1, y: -0.8, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.8}
destination:
  point: {x: 0.5, y: -2.0, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (493 bytes))
{"errors": [], "pose": {"angle": {"yaw": -0.8, "roll": 0.0, "pitch": 0.0}, "point": {"y": -0.8, "x": 0.1, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -2.0, "x": 0.5, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-08T16:59:06.720795+09:00"}
```

### 11. emulate the stop of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-08T14:05:00.000+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 3.71, y: -1.99, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -1.57}
destination:
  point: {x: 3.71, y: -1.99, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -1.57}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (538 bytes))
{"errors": [], "pose": {"angle": {"yaw": -1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.99, "x": 3.71, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.99, "x": 3.71, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-08T17:01:16.437847+09:00"}
```

#### confirm that a MQTT message to `/robot_ui/robotui_02/cmd` is sent automatically

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_02/cmd', ... (99 bytes))
{"send_state":{"time":"2019-11-08T17:01:16.596+09:00","state":"picking","destination":"事務室"}}
```

#### confirm log messages

example)
```
update robot state, robot_id=delivery_robot_02, current_mode=navi, next_mode=standby
publish new state to robot ui(robotui_02), current_state=moving, next_state=picking, destination=事務室
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is still locked by `delivery_robot_02`, and `delivery_robot_01` is now waiting for this token.
* `token_b` and `token_c` is not locked.

### 12. emulate the completion of  picking operation ( `delivery_robot_02`)

#### post a movenext data

In [ ]:
TOKEN=$(cat ~/core/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens | map(select(.allowed_paths[] | contains ("^/controller/.*$"))) | .[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Content-Type: application/json" https://api.tech-sketch.jp/controller/api/v1/robots/${DELIVERY_ROBOT_02}/nexts/ -X PATCH -d '{}'

example)

```
HTTP/1.1 201 Created
server: envoy
date: Fri, 08 Nov 2019 07:29:03 GMT
content-type: application/json
content-length: 150
access-control-allow-origin: *
x-envoy-upstream-service-time: 371

{"result":"success"}
```

#### confirm the MQTT message

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/cmd', ... (199 bytes))
{"send_cmd":{"time":"2019-11-08T17:09:31.644+09:00","cmd":"navi","waypoints":[{"point":{"x":0.5,"y":-2,"z":0},"angle":null},{"point":{"x":0,"y":-2.5,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-1.8}}]}}
```

#### confirm ros message for delivery_robot_02 ( `/robot_bridge/megarover_01/cmd` )

example)

```yaml
id: "delivery_robot_02"
type: "delivery_robot"
time: "2019-11-08T17:09:31.644+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 0.5
      y: -2.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 0.0
      y: -2.5
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -1.8
---
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is still locked by `delivery_robot_02`, and `delivery_robot_01` is still waiting for this token.
* `token_b` and `token_c` is not locked.

### 13. notify that the command is received

#### send a ros message

_On ROS Kinect PC_
1. start a terminal
1. execute below command.
```
rostopic pub -1 /robot_bridge/megarover_01/cmdexe uoa_poc2_msgs/r_result "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-08T14:06:00.000+09:00'
received_time: '2019-11-08T17:09:31.644+09:00'
received_cmd: 'navi'
received_waypoints:
  -
    point:
      x: 0.5
      y: -2.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 0.0
      y: -2.5
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -1.8
result: 'ack'
errors: ['']
"
```

#### confirm the MQTT message

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/cmdexe', ... (340 bytes))
{"send_cmd": {"received_time": "2019-11-08T17:09:31.644+09:00", "errors": [""], "received_cmd": "navi", "received_waypoints": [{"angle": null, "point": {"y": -2.0, "x": 0.5, "z": 0.0}}, {"angle": {"yaw": -1.8, "roll": 0.0, "pitch": 0.0}, "point": {"y": -2.5, "x": 0.0, "z": 0.0}}], "result": "ack", "time": "2019-11-08T14:06:00.000+09:00"}}
```

### 14. emulate the move of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-08T14:07:00.000+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 0.6, y: -1.9, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.01}
destination:
  point: {x: 0.5, y: -2.0, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (494 bytes))
{"errors": [], "pose": {"angle": {"yaw": -0.01, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.9, "x": 0.6, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -2.0, "x": 0.5, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-08T17:27:52.202297+09:00"}
```

#### confirm that a MQTT message to `/robot_ui/robotui_02/cmd` is sent automatically

example)
```
{"send_state":{"time":"2019-11-08T17:27:52.304+09:00","state":"moving","destination":"会議室1中"}}
```

#### confirm log messages

example)
```
update robot state, robot_id=delivery_robot_02, current_mode=standby, next_mode=navi
publish new state to robot ui(robotui_02), current_state=picking, next_state=moving, destination=会議室1中
```

### 15. emulate the stop of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-08T14:08:00.000+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 0.0, y: -2.5, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -1.8}
destination:
  point: {x: 0.0, y: -2.5, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -1.8}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (532 bytes))
{"errors": [], "pose": {"angle": {"yaw": -1.8, "roll": 0.0, "pitch": 0.0}, "point": {"y": -2.5, "x": 0.0, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -1.8, "roll": 0.0, "pitch": 0.0}, "point": {"y": -2.5, "x": 0.0, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-08T17:31:22.518981+09:00"}
```

#### confirm that a MQTT message to `/delivery_robot/delivery_robot_02/cmd` is sent automatically

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/cmd', ... (206 bytes))
{"send_cmd":{"time":"2019-11-08T17:31:22.763+09:00","cmd":"navi","waypoints":[{"point":{"x":-0.94,"y":-5.9,"z":0},"angle":null},{"point":{"x":0.7,"y":-7.4,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-0.15}}]}}
```

#### confirm that a ros message for delivery_robot_02 ( `/robot_bridge/megarover_01/cmd` ) is sent automatically

example)

```yaml
id: "delivery_robot_02"
type: "delivery_robot"
time: "2019-11-08T17:31:22.763+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: -0.94
      y: -5.9
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 0.7
      y: -7.4
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -0.15
---
```

#### confirm that a MQTT message to `/delivery_robot/delivery_robot_01/cmd` is sent automatically

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmd', ... (204 bytes))
{"send_cmd":{"time":"2019-11-08T17:31:22.928+09:00","cmd":"navi","waypoints":[{"point":{"x":0.5,"y":-2,"z":0},"angle":null},{"point":{"x":3.71,"y":-1.99,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-1.57}}]}}
```

#### confirm that a ros message for delivery_robot_01 ( `/robot_bridge/turtlebot_01/cmd` ) is sent automatically

example)

```yaml
id: "delivery_robot_01"
type: "delivery_robot"
time: "2019-11-08T17:31:22.928+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 0.5
      y: -2.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 3.71
      y: -1.99
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -1.57
---
```

#### confirm log messages

`delivery_robot_02` released `token_a`, so `delivery_robot_01` which is wating this token can lock it and can move into restlicted area.

example)

```
update robot state, robot_id=delivery_robot_02, current_mode=navi, next_mode=standby
switch token (token_a) from delivery_robot_02 to delivery_robot_01
move robot(delivery_robot_02) to "place_LB" (waypoints=[{'point': {'x': -0.94, 'y': -5.9, 'z': 0}, 'angle': None}, {'point': {'x': 0.7, 'y': -7.4, 'z': 0}, 'angle': {'roll': 0, 'pitch': 0, 'yaw': -0.15}}]
move robot(delivery_robot_01) to "place_L" (waypoints=[{'point': {'x': 0.5, 'y': -2, 'z': 0}, 'angle': None}, {'point': {'x': 3.71, 'y': -1.99, 'z': 0}, 'angle': {'roll': 0, 'pitch': 0, 'yaw': -1.57}}]
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is still locked, but its owner is switched from `delivery_robot_02` to `delivery_robot_01`, and `delivery_robot_01` is no longer waiting for this token.
* `token_b` and `token_c` is not locked.

### 16. emulate the move of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-08T14:09:00.000+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 0.1, y: -0.8, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.8}
destination:
  point: {x: 0.5, y: -2.0, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (493 bytes))
{"errors": [], "pose": {"angle": {"yaw": -0.8, "roll": 0.0, "pitch": 0.0}, "point": {"y": -0.8, "x": 0.1, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -2.0, "x": 0.5, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-08T17:45:24.386008+09:00"}
```

### 17. emulate the stop of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-08T14:10:00.000+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 3.71, y: -1.99, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -1.57}
destination:
  point: {x: 3.71, y: -1.99, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -1.57}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (538 bytes))
{"errors": [], "pose": {"angle": {"yaw": -1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.99, "x": 3.71, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.99, "x": 3.71, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-08T17:46:31.475726+09:00"}
```

#### confirm that a MQTT message to `/robot_ui/robotui_01/cmd` is sent automatically

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_01/cmd', ... (99 bytes))
{"send_state":{"time":"2019-11-08T17:46:31.640+09:00","state":"picking","destination":"事務室"}}
```

#### confirm log messages

example)
```
update robot state, robot_id=delivery_robot_01, current_mode=navi, next_mode=standby
publish new state to robot ui(robotui_01), current_state=moving, next_state=picking, destination=事務室
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is still locked by `delivery_robot_01`, and no robot is waiting for this token.
* token_b and token_c is not locked.

### 18. emulate the move of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-08T14:11:00.000+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: -0.90, y: -5.9, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 1.01}
destination:
  point: {x: -0.94, y: -5.9, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (496 bytes))
{"errors": [], "pose": {"angle": {"yaw": 1.01, "roll": 0.0, "pitch": 0.0}, "point": {"y": -5.9, "x": -0.9, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -5.9, "x": -0.94, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-08T17:53:29.923887+09:00"}
```

### 19. emulate the stop of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-08T14:08:00.000+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 0.7, y: -7.4, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.15}
destination:
  point: {x: 0.7, y: -7.4, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -0.15}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (534 bytes))
{"errors": [], "pose": {"angle": {"yaw": -0.15, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.4, "x": 0.7, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -0.15, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.4, "x": 0.7, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-08T17:54:55.198288+09:00"}
```

#### confirm that a MQTT message to `/delivery_robot/delivery_robot_02/cmd` is sent automatically

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/cmd', ... (203 bytes))
{"send_cmd":{"time":"2019-11-08T17:54:55.490+09:00","cmd":"navi","waypoints":[{"point":{"x":1.5,"y":-7.5,"z":0},"angle":null},{"point":{"x":2.3,"y":-7.5,"z":0},"angle":{"roll":0,"pitch":0,"yaw":0.05}}]}}
```

#### confirm that a ros message for delivery_robot_02 ( `/robot_bridge/megarover_01/cmd` ) is sent automatically

example)

```yaml
id: "delivery_robot_02"
type: "delivery_robot"
time: "2019-11-08T17:54:55.490+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 1.5
      y: -7.5
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 2.3
      y: -7.5
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.05
---
```

#### confirm log messages

example)

```
update robot state, robot_id=delivery_robot_02, current_mode=navi, next_mode=standby
lock token (token_b) by delivery_robot_02
move robot(delivery_robot_02) to "place_RB" (waypoints=[{'point': {'x': 1.5, 'y': -7.5, 'z': 0}, 'angle': None}, {'point': {'x': 2.3, 'y': -7.5, 'z': 0}, 'angle': {'roll': 0, 'pitch': 0, 'yaw': 0.05}}]
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is still locked by `delivery_robot_01`, and no robot is waiting for this token.
* now, `token_b` is locked by `delivery_robot_02`.
* `token_c` is not locked yet.

### 20. emulate the move of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-08T14:13:00.000+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: -0.4, y: -6.1, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.2}
destination:
  point: {x: 1.5, y: -7.5, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (494 bytes))
{"errors": [], "pose": {"angle": {"yaw": -0.2, "roll": 0.0, "pitch": 0.0}, "point": {"y": -6.1, "x": -0.4, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -7.5, "x": 1.5, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-08T18:06:35.669674+09:00"}
```

### 21. emulate the stop of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-08T14:14:00.000+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 2.3, y: -7.5, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 0.05}
destination:
  point: {x: 2.3, y: -7.5, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.05}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (532 bytes))
{"errors": [], "pose": {"angle": {"yaw": 0.05, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.5, "x": 2.3, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": 0.05, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.5, "x": 2.3, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-08T18:07:50.201604+09:00"}
```

#### confirm that a MQTT message to `/delivery_robot/delivery_robot_02/cmd` is sent automatically

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/cmd', ... (152 bytes))
{"send_cmd":{"time":"2019-11-08T18:07:50.376+09:00","cmd":"navi","waypoints":[{"point":{"x":4.9,"y":-7.3,"z":0},"angle":{"roll":0,"pitch":0,"yaw":0}}]}}
```

#### confirm that a ros message for delivery_robot_02 ( `/robot_bridge/megarover_01/cmd` ) is sent automatically

example)

```yaml
id: "delivery_robot_02"
type: "delivery_robot"
time: "2019-11-08T18:07:50.376+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 4.9
      y: -7.3
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
---
```

#### confirm log messages

example)

```
update robot state, robot_id=delivery_robot_02, current_mode=navi, next_mode=standby
release token (token_b) by delivery_robot_02
move robot(delivery_robot_02) to "place_LC" (waypoints=[{'point': {'x': 4.9, 'y': -7.3, 'z': 0}, 'angle': {'roll': 0, 'pitch': 0, 'yaw': 0}}]
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is still locked by `delivery_robot_01`, and no robot is waiting for this token.
* `token_b` is released.
* `token_c` is not locked yet.

### 22. emulate the move of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-08T14:13:00.000+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 2.4, y: -7.1, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
destination:
  point: {x: 4.9, y: -7.3, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (527 bytes))
{"errors": [], "pose": {"angle": {"yaw": 0.0, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.1, "x": 2.4, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": 0.0, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.3, "x": 4.9, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-08T18:16:07.311262+09:00"}
```

### 23. emulate the stop of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-08T14:16:00.000+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 4.9, y: -7.3, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
destination:
  point: {x: 4.9, y: -7.3, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (530 bytes))
{"errors": [], "pose": {"angle": {"yaw": 0.0, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.3, "x": 4.9, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": 0.0, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.3, "x": 4.9, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-08T18:20:49.349839+09:00"}
```

#### confirm that a MQTT message to `/delivery_robot/delivery_robot_02/cmd` is sent automatically

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/cmd', ... (300 bytes))
{"send_cmd":{"time":"2019-11-08T18:20:49.557+09:00","cmd":"navi","waypoints":[{"point":{"x":6.06,"y":-8,"z":0},"angle":null},{"point":{"x":9.56,"y":-8,"z":0},"angle":null},{"point":{"x":10.6,"y":-8,"z":0},"angle":null},{"point":{"x":10.73,"y":-9.08,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-1.57}}]}}
```

#### confirm that a ros message for delivery_robot_02 ( `/robot_bridge/megarover_01/cmd` ) is sent automatically

example)

```yaml
id: "delivery_robot_02"
type: "delivery_robot"
time: "2019-11-08T18:20:49.557+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 6.06
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 9.56
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 10.6
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 10.73
      y: -9.08
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -1.57
---
```

#### confirm log messages

example)

```
update robot state, robot_id=delivery_robot_02, current_mode=navi, next_mode=standby
lock token (token_c) by delivery_robot_02
move robot(delivery_robot_02) to "place_Jdash" (waypoints=[{'point': {'x': 6.06, 'y': -8, 'z': 0}, 'angle': None}, {'point': {'x': 9.56, 'y': -8, 'z': 0}, 'angle': None}, {'point': {'x': 10.6, 'y': -8, 'z': 0}, 'angle': None}, {'point': {'x': 10.73, 'y': -9.08, 'z': 0}, 'angle': {'roll': 0, 'pitch': 0, 'yaw': -1.57}}]
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is still locked by `delivery_robot_01`, and no robot is waiting for this token.
* `token_b` is not locked.
* `token_c` is locked by `delivery_robot_02`.

### 24. emulate the move of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-08T14:13:00.000+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 6.0, y: -7.1, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
destination:
  point: {x: 6.06, y: -8.0, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (528 bytes))
{"errors": [], "pose": {"angle": {"yaw": 0.0, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.1, "x": 6.0, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": 0.0, "roll": 0.0, "pitch": 0.0}, "point": {"y": -8.0, "x": 6.06, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-08T18:58:53.734401+09:00"}
```

### 25. emulate the stop of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-08T14:16:00.000+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 10.70, y: -9.01, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -1.56}
destination:
  point: {x: 10.73 , y: -9.08, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -1.57}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (539 bytes))
{"errors": [], "pose": {"angle": {"yaw": -1.56, "roll": 0.0, "pitch": 0.0}, "point": {"y": -9.01, "x": 10.7, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": -9.08, "x": 10.73, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-08T19:00:27.095651+09:00"}
```

#### confirm that a MQTT message to `/robot_ui/robotui_02/cmd` is sent automatically

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_02/cmd', ... (106 bytes))
{"send_state":{"time":"2019-11-08T19:00:27.237+09:00","state":"delivering","destination":"会議室1中"}}
```

#### confirm log messages

example)

```
update robot state, robot_id=delivery_robot_02, current_mode=navi, next_mode=standby
publish new state to robot ui(robotui_02), current_state=moving, next_state=delivering, destination=会議室1中
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is still locked by `delivery_robot_01`, and no robot is waiting for this token.
* `token_b` is not locked.
* `token_c` is still locked by `delivery_robot_02`.

### 26. emulate the completion of  picking operation ( `delivery_robot_01`)

#### post a movenext data

In [ ]:
TOKEN=$(cat ~/core/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens | map(select(.allowed_paths[] | contains ("^/controller/.*$"))) | .[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Content-Type: application/json" https://api.tech-sketch.jp/controller/api/v1/robots/${DELIVERY_ROBOT_01}/nexts/ -X PATCH -d '{}'

example)

```
HTTP/1.1 200 OK
server: envoy
date: Fri, 08 Nov 2019 10:23:42 GMT
content-type: application/json
content-length: 21
access-control-allow-origin: *
x-envoy-upstream-service-time: 131

{"result":"success"}
```

#### confirm the MQTT message

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmd', ... (199 bytes))
{"send_cmd":{"time":"2019-11-08T19:23:42.714+09:00","cmd":"navi","waypoints":[{"point":{"x":0.5,"y":-2,"z":0},"angle":null},{"point":{"x":0,"y":-2.5,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-1.8}}]}}
```

#### confirm ros message for delivery_robot_02 ( `/robot_bridge/megarover_01/cmd` )

example)

```yaml
id: "delivery_robot_01"
type: "delivery_robot"
time: "2019-11-08T19:23:42.714+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 0.5
      y: -2.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 0.0
      y: -2.5
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -1.8
---
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is still locked by `delivery_robot_01`.
* `token_b` is not locked.
* `token_c` is still locked by `delivery_robot_02`.

### 27. notify that the command is received

#### send a ros message

_On ROS Kinect PC_
1. start a terminal
1. execute below command.
```
rostopic pub -1 /robot_bridge/turtlebot_01/cmdexe uoa_poc2_msgs/r_result "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-08T14:06:00.000+09:00'
received_time: '2019-11-08T17:09:31.644+09:00'
received_cmd: 'navi'
received_waypoints:
  -
    point:
      x: 0.5
      y: -2.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 0.0
      y: -2.5
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -1.8
result: 'ack'
errors: ['']
"
```

#### confirm the MQTT message

example)
```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmdexe', ... (340 bytes))
{"send_cmd": {"received_time": "2019-11-08T19:23:42.714+09:00", "errors": [""], "received_cmd": "navi", "received_waypoints": [{"angle": null, "point": {"y": -2.0, "x": 0.5, "z": 0.0}}, {"angle": {"yaw": -1.8, "roll": 0.0, "pitch": 0.0}, "point": {"y": -2.5, "x": 0.0, "z": 0.0}}], "result": "ack", "time": "2019-11-08T14:17:00.000+09:00"}}
```

### 28. emulate the move of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-08T14:15:00.000+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 0.6, y: -1.9, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.01}
destination:
  point: {x: 0.5, y: -2.0, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (494 bytes))
{"errors": [], "pose": {"angle": {"yaw": -0.01, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.9, "x": 0.6, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -2.0, "x": 0.5, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-08T19:52:12.871777+09:00"}
```

#### confirm that a MQTT message to `/robot_ui/robotui_01/cmd` is sent automatically

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_01/cmd', ... (95 bytes))
{"send_state":{"time":"2019-11-08T19:52:12.978+09:00","state":"moving","destination":"個室"}}
```

#### confirm log messages

example)

```
update robot state, robot_id=delivery_robot_01, current_mode=standby, next_mode=navi
publish new state to robot ui(robotui_01), current_state=picking, next_state=moving, destination=個室
```

### 29. emulate the stop of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-08T14:16:00.000+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 0.0, y: -2.5, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -1.8}
destination:
  point: {x: 0.0, y: -2.5, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -1.8}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (532 bytes))
{"errors": [], "pose": {"angle": {"yaw": -1.8, "roll": 0.0, "pitch": 0.0}, "point": {"y": -2.5, "x": 0.0, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -1.8, "roll": 0.0, "pitch": 0.0}, "point": {"y": -2.5, "x": 0.0, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-08T20:00:24.452597+09:00"}
```

#### confirm that a MQTT message to `/delivery_robot/delivery_robot_01/cmd` is sent automatically

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmd', ... (206 bytes))
{"send_cmd":{"time":"2019-11-08T20:00:24.703+09:00","cmd":"navi","waypoints":[{"point":{"x":-0.94,"y":-5.9,"z":0},"angle":null},{"point":{"x":0.7,"y":-7.4,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-0.15}}]}}
```

#### confirm that a ros message for delivery_robot_01 ( `/robot_bridge/turtlebot_01/cmd` ) is sent automatically

example)

```yaml
id: "delivery_robot_01"
type: "delivery_robot"
time: "2019-11-08T20:00:24.703+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: -0.94
      y: -5.9
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 0.7
      y: -7.4
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -0.15
---
```

#### confirm log messages

example)

```
update robot state, robot_id=delivery_robot_01, current_mode=navi, next_mode=standby
release token (token_a) by delivery_robot_01
move robot(delivery_robot_01) to "place_LB" (waypoints=[{'point': {'x': -0.94, 'y': -5.9, 'z': 0}, 'angle': None}, {'point': {'x': 0.7, 'y': -7.4, 'z': 0}, 'angle': {'roll': 0, 'pitch': 0, 'yaw': -0.15}}]
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` has been released.
* `token_b` is not locked.
* `token_c` is still locked by `delivery_robot_02`.

### 30. emulate the move of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-08T14:17:00.000+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: -0.90, y: -5.9, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 1.01}
destination:
  point: {x: -0.94, y: -5.9, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (496 bytes))
{"errors": [], "pose": {"angle": {"yaw": 1.01, "roll": 0.0, "pitch": 0.0}, "point": {"y": -5.9, "x": -0.9, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -5.9, "x": -0.94, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-08T20:26:19.333110+09:00"}
```

### 31. emulate the stop of  `delivery_robot_01`

#### send a ros message

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-08T14:18:00.000+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 0.7, y: -7.4, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.15}
destination:
  point: {x: 0.7, y: -7.4, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -0.15}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (534 bytes))
{"errors": [], "pose": {"angle": {"yaw": -0.15, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.4, "x": 0.7, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -0.15, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.4, "x": 0.7, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-08T20:33:29.256345+09:00"}
```

#### confirm that a MQTT message to `/delivery_robot/delivery_robot_01/cmd` is sent automatically

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmd', ... (205 bytes))
{"send_cmd":{"time":"2019-11-08T20:33:29.512+09:00","cmd":"navi","waypoints":[{"point":{"x":1.5,"y":-7.5,"z":0},"angle":null},{"point":{"x":1.51,"y":-9.7,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-1.57}}]}}
```

#### confirm that a ros message for delivery_robot_01 ( `/robot_bridge/turtlebot_01/cmd` ) is sent automatically

example)

```yaml
id: "delivery_robot_01"
type: "delivery_robot"
time: "2019-11-08T20:33:29.512+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 1.5
      y: -7.5
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 1.51
      y: -9.7
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -1.57
---
```

#### confirm log messages

example)

```
update robot state, robot_id=delivery_robot_01, current_mode=navi, next_mode=standby
lock token (token_b) by delivery_robot_01
move robot(delivery_robot_01) to "place_G" (waypoints=[{'point': {'x': 1.5, 'y': -7.5, 'z': 0}, 'angle': None}, {'point': {'x': 1.51, 'y': -9.7, 'z': 0}, 'angle': {'roll': 0, 'pitch': 0, 'yaw': -1.57}}]
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is not locked.
* now, `token_b` has been locked by `delivery_robot_01`.
* `token_c` is locked by `delivery_robot_02`.

### 32. emulate the move of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-08T14:11:00.000+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 1.4, y: -5.1, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 1.01}
destination:
  point: {x: 1.5, y: -7.5, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (493 bytes))
{"errors": [], "pose": {"angle": {"yaw": 1.01, "roll": 0.0, "pitch": 0.0}, "point": {"y": -5.1, "x": 1.4, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -7.5, "x": 1.5, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-08T20:52:43.153145+09:00"}
```

### 33. emulate the stop of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-08T14:19:00.000+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 1.51, y: -9.7, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -1.57}
destination:
  point: {x: 1.51, y: -9.7, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -1.57}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (536 bytes))
{"errors": [], "pose": {"angle": {"yaw": -1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": -9.7, "x": 1.51, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": -9.7, "x": 1.51, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-08T20:55:45.983433+09:00"}
```

#### confirm that a MQTT message to `/robot_ui/robotui_01/cmd` is sent automatically

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_01/cmd', ... (96 bytes))
{"send_state":{"time":"2019-11-08T20:55:46.257+09:00","state":"picking","destination":"個室"}}
```

#### confirm log messages

example)

```
update robot state, robot_id=delivery_robot_01, current_mode=navi, next_mode=standby
publish new state to robot ui(robotui_01), current_state=moving, next_state=picking, destination=個室
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is not locked.
* `token_b` is locked by `delivery_robot_01`.
* `token_c` is locked by `delivery_robot_02`.

### 34. emulate the completion of  picking operation ( `delivery_robot_01`)

#### post a movenext data

In [ ]:
TOKEN=$(cat ~/core/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens | map(select(.allowed_paths[] | contains ("^/controller/.*$"))) | .[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Content-Type: application/json" https://api.tech-sketch.jp/controller/api/v1/robots/${DELIVERY_ROBOT_01}/nexts/ -X PATCH -d '{}'

example)

```
HTTP/1.1 200 OK
server: envoy
date: Fri, 08 Nov 2019 12:02:36 GMT
content-type: application/json
content-length: 21
access-control-allow-origin: *
x-envoy-upstream-service-time: 107

{"result":"success"}
```

#### confirm the MQTT message

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmd', ... (203 bytes))
{"send_cmd":{"time":"2019-11-08T21:02:36.428+09:00","cmd":"navi","waypoints":[{"point":{"x":1.5,"y":-7.5,"z":0},"angle":null},{"point":{"x":2.3,"y":-7.5,"z":0},"angle":{"roll":0,"pitch":0,"yaw":0.05}}]}}
```

#### confirm ros message for delivery_robot_02 ( `/robot_bridge/megarover_01/cmd` )

example)

```yaml
id: "delivery_robot_01"
type: "delivery_robot"
time: "2019-11-08T21:02:36.428+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 1.5
      y: -7.5
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 2.3
      y: -7.5
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.05
---
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is not locked.
* `token_b` is locked by `delivery_robot_01`.
* `token_c` is locked by `delivery_robot_02`.

### 35. notify that the command is received

#### send a ros message

_On ROS Kinect PC_
1. start a terminal
1. execute below command.
```
rostopic pub -1 /robot_bridge/turtlebot_01/cmdexe uoa_poc2_msgs/r_result "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-08T14:20:00.000+09:00'
received_time: '2019-11-08T21:02:36.428+09:00'
received_cmd: 'navi'
received_waypoints:
  -
    point:
      x: 1.5
      y: -7.5
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 2.3
      y: -7.5
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.05
result: 'ack'
errors: ['']
"
```

#### confirm the MQTT message

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmdexe', ... (340 bytes))
{"send_cmd": {"received_time": "2019-11-08T21:02:36.428+09:00", "errors": [""], "received_cmd": "navi", "received_waypoints": [{"angle": null, "point": {"y": -7.5, "x": 1.5, "z": 0.0}}, {"angle": {"yaw": 0.05, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.5, "x": 2.3, "z": 0.0}}], "result": "ack", "time": "2019-11-08T14:20:00.000+09:00"}}
```

### 36. emulate the move of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-08T14:21:00.000+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 1.4, y: -6.1, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 1.01}
destination:
  point: {x: 1.5, y: -7.5, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (493 bytes))
{"errors": [], "pose": {"angle": {"yaw": 1.01, "roll": 0.0, "pitch": 0.0}, "point": {"y": -6.1, "x": 1.4, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -7.5, "x": 1.5, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-08T21:10:57.831685+09:00"}
```

#### confirm that a MQTT message to `/robot_ui/robotui_01/cmd` is sent automatically

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_01/cmd', ... (102 bytes))
{"send_state":{"time":"2019-11-08T21:10:57.928+09:00","state":"moving","destination":"会議室2中"}}
```

#### confirm log messages

example)

```
update robot state, robot_id=delivery_robot_01, current_mode=standby, next_mode=navi
publish new state to robot ui(robotui_01), current_state=picking, next_state=moving, destination=会議室2中
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is not locked.
* `token_b` is locked by `delivery_robot_01`.
* `token_c` is locked by `delivery_robot_02`.

### 37. emulate the stop of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-08T14:08:00.000+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 2.3, y: -7.5, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 0.05}
destination:
  point: {x: 2.3, y: -7.5, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.05}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (532 bytes))
{"errors": [], "pose": {"angle": {"yaw": 0.05, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.5, "x": 2.3, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": 0.05, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.5, "x": 2.3, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-08T21:16:15.591016+09:00"}
```

#### confirm that a MQTT message to `/delivery_robot/delivery_robot_01/cmd` is sent automatically

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmd', ... (152 bytes))
{"send_cmd":{"time":"2019-11-08T21:16:15.858+09:00","cmd":"navi","waypoints":[{"point":{"x":4.9,"y":-7.3,"z":0},"angle":{"roll":0,"pitch":0,"yaw":0}}]}}
```

#### confirm that a ros message for delivery_robot_02 ( `/robot_bridge/megarover_01/cmd` ) is sent automatically

example)

```yaml
id: "delivery_robot_01"
type: "delivery_robot"
time: "2019-11-08T21:16:15.858+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 4.9
      y: -7.3
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
---
```

#### confirm log messages

example)

```
update robot state, robot_id=delivery_robot_01, current_mode=navi, next_mode=standby
release token (token_b) by delivery_robot_01
move robot(delivery_robot_01) to "place_LC" (waypoints=[{'point': {'x': 4.9, 'y': -7.3, 'z': 0}, 'angle': {'roll': 0, 'pitch': 0, 'yaw': 0}}]
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is not locked.
* `token_b` has been released.
* `token_c` is locked by `delivery_robot_02`.

### 38. emulate the move of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-08T14:22:00.000+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 4.1, y: -7.0, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
destination:
  point: {x: 4.9, y: -7.3, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (527 bytes))
{"errors": [], "pose": {"angle": {"yaw": 0.0, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.0, "x": 4.1, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": 0.0, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.3, "x": 4.9, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-08T21:21:47.964833+09:00"}
```

### 39. emulate the stop of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-08T14:08:00.000+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 4.9, y: -7.3, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
destination:
  point: {x: 4.9, y: -7.3, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (530 bytes))
{"errors": [], "pose": {"angle": {"yaw": 0.0, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.3, "x": 4.9, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": 0.0, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.3, "x": 4.9, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-08T21:24:22.272196+09:00"}
```

#### confirm that a MQTT message to `/delivery_robot/delivery_robot_01/cmd` is sent automatically

`delivery_robot_01` can not lock `token_c`, so `delivery_robot_01` has to take refuge in 'place_U'

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmd', ... (157 bytes))
{"send_cmd":{"time":"2019-11-08T21:24:22.508+09:00","cmd":"navi","waypoints":[{"point":{"x":5.3,"y":-7.15,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-1.57}}]}}
```

#### confirm that a ros message for delivery_robot_02 ( `/robot_bridge/megarover_01/cmd` ) is sent automatically

example)

```yaml
id: "delivery_robot_01"
type: "delivery_robot"
time: "2019-11-08T21:24:22.508+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 5.3
      y: -7.15
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -1.57
---
```

#### confirm log messages

example)

```
update robot state, robot_id=delivery_robot_01, current_mode=navi, next_mode=standby
wait token (token_c) by delivery_robot_01
take refuge a robot(delivery_robot_01) in "place_U"
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is not locked.
* `token_b` is not locked.
* `token_c` is still locked by `delivery_robot_02`, and `delivery_robot_01` has been waiting for this token.

### 40. emulate the completion of delivering operation ( `delivery_robot_02`)

#### post a movenext data

In [ ]:
TOKEN=$(cat ~/core/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens | map(select(.allowed_paths[] | contains ("^/controller/.*$"))) | .[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Content-Type: application/json" https://api.tech-sketch.jp/controller/api/v1/robots/${DELIVERY_ROBOT_02}/nexts/ -X PATCH -d '{}'

example)

```
HTTP/1.1 200 OK
server: envoy
date: Fri, 08 Nov 2019 12:33:51 GMT
content-type: application/json
content-length: 21
access-control-allow-origin: *
x-envoy-upstream-service-time: 121

{"result":"success"}
```

#### confirm the MQTT message

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/cmd', ... (292 bytes))
{"send_cmd":{"time":"2019-11-08T21:33:51.025+09:00","cmd":"navi","waypoints":[{"point":{"x":10.6,"y":-8,"z":0},"angle":null},{"point":{"x":9.56,"y":-8,"z":0},"angle":null},{"point":{"x":6.06,"y":-8,"z":0},"angle":null},{"point":{"x":4,"y":-8,"z":0},"angle":{"roll":0,"pitch":0,"yaw":3.14}}]}}
```

#### confirm ros message for delivery_robot_02 ( `/robot_bridge/megarover_01/cmd` )

example)

```yaml
id: "delivery_robot_02"
type: "delivery_robot"
time: "2019-11-08T21:33:51.025+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 10.6
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 9.56
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 6.06
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 4.0
      y: -8.0
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 3.14
---
```

### 41. notify that the command is received

#### send a ros message

_On ROS Kinect PC_
1. start a terminal
1. execute below command.
```
rostopic pub -1 /robot_bridge/megarover_01/cmdexe uoa_poc2_msgs/r_result "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-08T14:23:00.000+09:00'
received_time: '2019-11-08T21:33:51.025+09:00'
received_cmd: 'navi'
received_waypoints:
  -
    point:
      x: 10.6
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 9.56
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 6.06
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 4.0
      y: -8.0
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 3.14
result: 'ack'
errors: ['']
"
```

#### confirm the MQTT message

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/cmdexe', ... (461 bytes))
{"send_cmd": {"received_time": "2019-11-08T21:33:51.025+09:00", "errors": [""], "received_cmd": "navi", "received_waypoints": [{"angle": null, "point": {"y": -8.0, "x": 10.6, "z": 0.0}}, {"angle": null, "point": {"y": -8.0, "x": 9.56, "z": 0.0}}, {"angle": null, "point": {"y": -8.0, "x": 6.06, "z": 0.0}}, {"angle": {"yaw": 3.14, "roll": 0.0, "pitch": 0.0}, "point": {"y": -8.0, "x": 4.0, "z": 0.0}}], "result": "ack", "time": "2019-11-08T14:23:00.000+09:00"}}
```

### 42. emulate the move of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-08T14:24:00.000+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 10.1, y: -7.1, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 1.5}
destination:
  point: {x: 10.6, y: -8.0, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (494 bytes))
{"errors": [], "pose": {"angle": {"yaw": 1.5, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.1, "x": 10.1, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -8.0, "x": 10.6, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-08T21:41:55.111182+09:00"}
```

#### confirm that a MQTT message to `/robot_ui/robotui_02/cmd` is sent automatically

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_02/cmd', ... (104 bytes))
{"send_state":{"time":"2019-11-08T21:41:55.240+09:00","state":"moving","destination":"待機場所２"}}
```

#### confirm log messages

example)

```
update robot state, robot_id=delivery_robot_02, current_mode=standby, next_mode=navi
publish new state to robot ui(robotui_02), current_state=delivering, next_state=moving, destination=待機場所２
```

### 43. emulate the stop of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-08T14:25:00.000+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 4.0, y: -8.0, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 3.14}
destination:
  point: {x: 4.0, y: -8.0, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: 3.14}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (532 bytes))
{"errors": [], "pose": {"angle": {"yaw": 3.14, "roll": 0.0, "pitch": 0.0}, "point": {"y": -8.0, "x": 4.0, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": 3.14, "roll": 0.0, "pitch": 0.0}, "point": {"y": -8.0, "x": 4.0, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-08T21:46:08.407250+09:00"}
```

#### confirm that a MQTT message to `/delivery_robot/delivery_robot_02/cmd` is sent automatically

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/cmd', ... (302 bytes))
{"send_cmd":{"time":"2019-11-08T21:46:08.662+09:00","cmd":"navi","waypoints":[{"point":{"x":0.2,"y":-8.1,"z":0},"angle":null},{"point":{"x":-1.66,"y":-6.68,"z":0},"angle":null},{"point":{"x":-1,"y":-2.5,"z":0},"angle":null},{"point":{"x":-1.5,"y":1.5,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-1.57}}]}}
```

#### confirm that a ros message for delivery_robot_02 ( `/robot_bridge/megarover_01/cmd` ) is sent automatically

example)

```yaml
id: "delivery_robot_02"
type: "delivery_robot"
time: "2019-11-08T21:46:08.662+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 0.2
      y: -8.1
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: -1.66
      y: -6.68
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: -1.0
      y: -2.5
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: -1.5
      y: 1.5
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -1.57
---
```

#### confirm that a MQTT message to `/delivery_robot/delivery_robot_01/cmd` is sent automatically

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmd', ... (251 bytes))
{"send_cmd":{"time":"2019-11-08T21:46:08.771+09:00","cmd":"navi","waypoints":[{"point":{"x":6.06,"y":-8,"z":0},"angle":null},{"point":{"x":9.56,"y":-8,"z":0},"angle":null},{"point":{"x":9.43,"y":-9.1,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-1.57}}]}}
```

#### confirm that a ros message for `delivery_robot_01` ( `/robot_bridge/turtlebot_01/cmd` ) is sent automatically

example)

```yaml
id: "delivery_robot_01"
type: "delivery_robot"
time: "2019-11-08T21:46:08.771+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 6.06
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 9.56
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 9.43
      y: -9.1
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -1.57
---
```

#### confirm log messages

`delivery_robot_02` released `token_c`, so `delivery_robot_01` which is wating this token can lock it and can move into restlicted area.

example)

```
update robot state, robot_id=delivery_robot_02, current_mode=navi, next_mode=standby
switch token (token_c) from delivery_robot_02 to delivery_robot_01
move robot(delivery_robot_02) to "place_V" (waypoints=[{'point': {'x': 0.2, 'y': -8.1, 'z': 0}, 'angle': None}, {'point': {'x': -1.66, 'y': -6.68, 'z': 0}, 'angle': None}, {'point': {'x': -1, 'y': -2.5, 'z': 0}, 'angle': None}, {'point': {'x': -1.5, 'y': 1.5, 'z': 0}, 'angle': {'roll': 0, 'pitch': 0, 'yaw': -1.57}}]
move robot(delivery_robot_01) to "place_Idash" (waypoints=[{'point': {'x': 6.06, 'y': -8, 'z': 0}, 'angle': None}, {'point': {'x': 9.56, 'y': -8, 'z': 0}, 'angle': None}, {'point': {'x': 9.43, 'y': -9.1, 'z': 0}, 'angle': {'roll': 0, 'pitch': 0, 'yaw': -1.57}}]
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` and `token_b` is not locked.
* `token_c` is still locked, but its owner is switched from `delivery_robot_02` to `delivery_robot_01`, and `delivery_robot_01` is no longer waiting for this token.

### 44. emulate the move of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-08T14:26:00.000+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 0.1, y: -8.0, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
destination:
  point: {x: 0.2, y: -8.1, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (492 bytes))
{"errors": [], "pose": {"angle": {"yaw": 0.0, "roll": 0.0, "pitch": 0.0}, "point": {"y": -8.0, "x": 0.1, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -8.1, "x": 0.2, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-08T21:58:55.151508+09:00"}
```

### 45. emulate the stop of  `delivery_robot_02`

#### send a ros message

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-08T14:27:00.000+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: -1.5, y: 1.5, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -1.57}
destination:
  point: {x: -1.5, y: 1.5, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -1.57}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (534 bytes))
{"errors": [], "pose": {"angle": {"yaw": -1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": 1.5, "x": -1.5, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": 1.5, "x": -1.5, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-08T22:02:16.924000+09:00"}
```

#### confirm that a MQTT message to `/robot_ui/robotui_02/cmd` is sent automatically

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_02/cmd', ... (105 bytes))
{"send_state":{"time":"2019-11-08T22:02:17.078+09:00","state":"standby","destination":"待機場所２"}}
```

#### confirm log messages

example)

```
update robot state, robot_id=delivery_robot_02, current_mode=navi, next_mode=standby
publish new state to robot ui(robotui_02), current_state=moving, next_state=standby, destination=待機場所２
```

#### confirm the `delivery_robot_02` entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" "https://api.${DOMAIN}/orion/v2/entities/${DELIVERY_ROBOT_02}?type=${DELIVERY_ROBOT_TYPE}&attrs=remaining_waypoints_list" | jq .

confirm below:

* `remaining_waypoints_list` is empty list

### 46. emulate the move of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-08T14:11:00.000+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 6.06, y: -8.0, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 1.01}
destination:
  point: {x: 6.06, y: -8.0, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (495 bytes))
{"errors": [], "pose": {"angle": {"yaw": 1.01, "roll": 0.0, "pitch": 0.0}, "point": {"y": -8.0, "x": 6.06, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -8.0, "x": 6.06, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-08T22:11:40.621717+09:00"}
```

### 47. emulate the stop of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-08T14:29:00.000+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 9.43, y: -9.1, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -1.57}
destination:
  point: {x: 9.43, y: -9.1, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -1.57}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (536 bytes))
{"errors": [], "pose": {"angle": {"yaw": -1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": -9.1, "x": 9.43, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": -9.1, "x": 9.43, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-08T22:14:31.767731+09:00"}
```

#### confirm that a MQTT message to `/robot_ui/robotui_01/cmd` is sent automatically

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_01/cmd', ... (106 bytes))
{"send_state":{"time":"2019-11-08T22:14:31.910+09:00","state":"delivering","destination":"会議室2中"}}
```

#### confirm log messages

example)

```
update robot state, robot_id=delivery_robot_01, current_mode=navi, next_mode=standby
publish new state to robot ui(robotui_01), current_state=moving, next_state=delivering, destination=会議室2中
```

### 48. emulate the completion of delivering operation ( `delivery_robot_01`)

#### post a movenext data

In [ ]:
TOKEN=$(cat ~/core/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens | map(select(.allowed_paths[] | contains ("^/controller/.*$"))) | .[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Content-Type: application/json" https://api.tech-sketch.jp/controller/api/v1/robots/${DELIVERY_ROBOT_01}/nexts/ -X PATCH -d '{}'

example)

```
HTTP/1.1 200 OK
server: envoy
date: Fri, 08 Nov 2019 13:18:24 GMT
content-type: application/json
content-length: 21
access-control-allow-origin: *
x-envoy-upstream-service-time: 124

{"result":"success"}
```

#### confirm the MQTT message

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmd', ... (245 bytes))
{"send_cmd":{"time":"2019-11-08T22:18:24.472+09:00","cmd":"navi","waypoints":[{"point":{"x":9.56,"y":-8,"z":0},"angle":null},{"point":{"x":6.06,"y":-8,"z":0},"angle":null},{"point":{"x":4,"y":-8,"z":0},"angle":{"roll":0,"pitch":0,"yaw":3.14}}]}}
```

#### confirm ros message for delivery_robot_01 ( `/robot_bridge/turtlebot_01/cmd` )

example)

```yaml
id: "delivery_robot_01"
type: "delivery_robot"
time: "2019-11-08T22:18:24.472+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 9.56
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 6.06
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 4.0
      y: -8.0
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 3.14
---
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` and `token_b` is not locked.
* `token_c` is still locked by `delivery_robot_01`.

### 49. notify that the command is received

#### send a ros message

_On ROS Kinect PC_
1. start a terminal
1. execute below command.
```
rostopic pub -1 /robot_bridge/turtlebot_01/cmdexe uoa_poc2_msgs/r_result "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-08T14:29:00.000+09:00'
received_time: '2019-11-08T22:18:24.472+09:00'
received_cmd: 'navi'
received_waypoints:
  -
    point:
      x: 9.56
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 6.06
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 4.0
      y: -8.0
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 3.14
result: 'ack'
errors: ['']
"
```

#### confirm the MQTT message

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmdexe', ... (401 bytes))
{"send_cmd": {"received_time": "2019-11-08T22:18:24.472+09:00", "errors": [""], "received_cmd": "navi", "received_waypoints": [{"angle": null, "point": {"y": -8.0, "x": 9.56, "z": 0.0}}, {"angle": null, "point": {"y": -8.0, "x": 6.06, "z": 0.0}}, {"angle": {"yaw": 3.14, "roll": 0.0, "pitch": 0.0}, "point": {"y": -8.0, "x": 4.0, "z": 0.0}}], "result": "ack", "time": "2019-11-08T14:29:00.000+09:00"}}
```

### 50. emulate the move of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-08T14:30:00.000+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 9.56, y: -8.0, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 1.5}
destination:
  point: {x: 9.56, y: -8.0, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (494 bytes))
{"errors": [], "pose": {"angle": {"yaw": 1.5, "roll": 0.0, "pitch": 0.0}, "point": {"y": -8.0, "x": 9.56, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -8.0, "x": 9.56, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-08T22:32:11.755742+09:00"}
```

#### confirm that a MQTT message to `/robot_ui/robotui_01/cmd` is sent automatically

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_01/cmd', ... (104 bytes))
{"send_state":{"time":"2019-11-08T22:32:11.863+09:00","state":"moving","destination":"待機場所１"}}
```

#### confirm log messages

example)

```
update robot state, robot_id=delivery_robot_01, current_mode=standby, next_mode=navi
publish new state to robot ui(robotui_01), current_state=delivering, next_state=moving, destination=待機場所１
```

### 51. emulate the stop of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-08T14:32:00.000+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 4.0, y: -8.0, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 3.14}
destination:
  point: {x: 4.0, y: -8.0, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: 3.14}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (532 bytes))
{"errors": [], "pose": {"angle": {"yaw": 3.14, "roll": 0.0, "pitch": 0.0}, "point": {"y": -8.0, "x": 4.0, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": 3.14, "roll": 0.0, "pitch": 0.0}, "point": {"y": -8.0, "x": 4.0, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-08T22:38:05.115714+09:00"}
```

#### confirm that a MQTT message to `/delivery_robot/delivery_robot_01/cmd` is sent automatically

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmd', ... (302 bytes))
{"send_cmd":{"time":"2019-11-08T22:38:05.314+09:00","cmd":"navi","waypoints":[{"point":{"x":0.2,"y":-8.1,"z":0},"angle":null},{"point":{"x":-1.66,"y":-6.68,"z":0},"angle":null},{"point":{"x":-1,"y":-2.5,"z":0},"angle":null},{"point":{"x":-0.4,"y":1.5,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-1.57}}]}}
```

#### confirm that a ros message for delivery_robot_01 ( `/robot_bridge/turtlebot_01/cmd` ) is sent automatically

example)

```yaml
id: "delivery_robot_01"
type: "delivery_robot"
time: "2019-11-08T22:38:05.314+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 0.2
      y: -8.1
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: -1.66
      y: -6.68
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: -1.0
      y: -2.5
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: -0.4
      y: 1.5
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -1.57
---
```

#### confirm log messages

example)

```
update robot state, robot_id=delivery_robot_01, current_mode=navi, next_mode=standby
release token (token_c) by delivery_robot_01
move robot(delivery_robot_01) to "place_Q" (waypoints=[{'point': {'x': 0.2, 'y': -8.1, 'z': 0}, 'angle': None}, {'point': {'x': -1.66, 'y': -6.68, 'z': 0}, 'angle': None}, {'point': {'x': -1, 'y': -2.5, 'z': 0}, 'angle': None}, {'point': {'x': -0.4, 'y': 1.5, 'z': 0}, 'angle': {'roll': 0, 'pitch': 0, 'yaw': -1.57}}]
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* all token is not locked.

### 52. emulate the move of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-08T14:33:00.000+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 0.1, y: -8.0, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
destination:
  point: {x: 0.2, y: -8.1, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (492 bytes))
{"errors": [], "pose": {"angle": {"yaw": 0.0, "roll": 0.0, "pitch": 0.0}, "point": {"y": -8.0, "x": 0.1, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -8.1, "x": 0.2, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-08T22:45:45.445452+09:00"}
```

### 53. emulate the stop of  `delivery_robot_01`

#### send a ros message

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-08T14:34:00.000+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: -0.4, y: 1.5, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -1.57}
destination:
  point: {x: -0.4, y: 1.5, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -1.57}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (534 bytes))
{"errors": [], "pose": {"angle": {"yaw": -1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": 1.5, "x": -0.4, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": 1.5, "x": -0.4, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-08T22:49:49.231322+09:00"}
```

#### confirm that a MQTT message to `/robot_ui/robotui_01/cmd` is sent automatically

example)

```
Client mosq-YfCnDRG4u3kDoHupoy received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_01/cmd', ... (105 bytes))
{"send_state":{"time":"2019-11-08T22:49:49.418+09:00","state":"standby","destination":"待機場所１"}}
```

#### confirm log messages

example)

```
update robot state, robot_id=delivery_robot_01, current_mode=navi, next_mode=standby
publish new state to robot ui(robotui_01), current_state=moving, next_state=standby, destination=待機場所１
```

#### confirm the `delivery_robot_02` entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" "https://api.${DOMAIN}/orion/v2/entities/${DELIVERY_ROBOT_01}?type=${DELIVERY_ROBOT_TYPE}&attrs=remaining_waypoints_list" | jq .

confirm below:

* `remaining_waypoints_list` is empty list